In [1]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation

from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy import Function
from sympy import Matrix

In [2]:
edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64) 
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",") 
n = len(P)
L =np.genfromtxt('csv/eigVals.csv', delimiter=",") 
L_pos = np.array([L[i] if L[i]>0 else 0 for i in range(n)])
d = np.count_nonzero(L_pos)	# d ... the number of positive values
Ln = np.sqrt(L_pos)

f2 = np.array(Ln[0:d])
f2[::2] = 0
f1 = Ln[0:d] - f2
e1 = f1 / np.linalg.norm(f1)
e2 = f2 / np.linalg.norm(f2)
temp1 = e1
temp2 = e2

Xs = np.array([])
Ys = np.array([])

for i in np.arange(n):
    p0 = P[i,0:d]
    Xs = np.append(Xs,np.dot(p0,e1))
    Ys = np.append(Ys,np.dot(p0,e2))
    
    
# V:sp.Matrix(MatrixSymbol), Vv : np.ndarray
def subV(e, V, Vv):
    shape = Vv.shape
    for i in range(len(Vv)):
            e = e.subs(V[i,0], Vv[i])
    return e
# Vs, Vvs : list
def subV_mul(e,Vs,Vvs):
    if len(Vvs) > 0:
        if isinstance(Vvs[0], np.ndarray):
            return subV_mul(subV(e,Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
        else:
            return subV_mul(e.subs(Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
    return e

In [3]:
sp.var('a1 b1 c1 a2 b2 c2 t s')   # variables
sp.var('x2_s y2_s p_s e1_s e2_s') # values
p_s = sp.MatrixSymbol('p_s', d, 1)
e1_s = sp.MatrixSymbol('e1_s', d, 1)
e2_s = sp.MatrixSymbol('e2_s', d, 1)
p_s = sp.Matrix(p_s)
e1_s = sp.Matrix(e1_s)
e2_s = sp.Matrix(e2_s)
var = (a1,b1,c1,a2,b2,c2,t,s)
v = [x2_s, y2_s, p_s, e1_s, e2_s]
vv = [Xs[0],Ys[0],P[0],e1,e2]
E1 = a1*sp.Matrix(e1_s) + b1*sp.Matrix(e2_s) + c1*sp.Matrix(p_s)
E2 = a2*sp.Matrix(e1_s) + b2*sp.Matrix(e2_s) + c2*sp.Matrix(p_s)
r  = s*sp.Matrix(e1_s) + t*sp.Matrix(e2_s)

f = Matrix([
		E1.dot(E1) - 1,
		E2.dot(E2) - 1,
		E1.dot(E2),
		r.dot(r) - 1,
		E1.dot(r) - e1_s.dot(r),
		E2.dot(r) - e2_s.dot(r),
		p_s.dot(E1) - x2_s,
		p_s.dot(E2) - y2_s
		])

func = Matrix.norm(f)
lam_f = lambdify(var, subV_mul(func,v,vv), 'numpy')

arr = np.array([1,1,1,1,1,1,1,1])
def g(args): return lam_f(*args)

In [4]:
identifier = ""

class DraggableCircle:
    def __init__(self, circle):
        self.circle = circle
        self.press = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.circle.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.circle.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.circle.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.circle.axes: return

        contains, attrd = self.circle.contains(event)
        if not contains: return
        print('event contains', self.circle.center)
        x0, y0 = self.circle.center
        self.press = x0, y0, event.xdata, event.ydata
        global identifier
        identifier = self.circle.get_label()

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if self.press is None: return
        if event.inaxes != self.circle.axes: return
        if(self.circle.get_label() == identifier):
            global p, e1, e2, temp1, temp2, vv
            x0, y0, xpress, ypress = self.press
            x2 = x0 + event.xdata - xpress
            y2 = y0 + event.ydata - ypress
            vv = [x2,y2,P[int(identifier)],e1,e2]
            lam_f = lambdify(var, subV_mul(func,v,vv), 'numpy')
            def g(args): return lam_f(*args)
            res = opt.minimize(g,arr,method='L-BFGS-B')
            e1 = res.x[0]*temp1 + res.x[1]*temp2 + res.x[2]*p
            e2 = res.x[3]*temp1 + res.x[4]*temp2 + res.x[5]*p
            temp1 = e1
            temp2 = e2
            for i in np.arange(n):
                p0 = P[i]
                global Xs, Ys
                Xs[i] = np.dot(p0,e1)
                Ys[i] = np.dot(p0,e2)
                nodes[i].center = (np.dot(p0,e1),np.dot(p0,e2))
                nodes[i].figure.canvas.draw()
                        
    def on_release(self, event):
        'on release we reset the press data'
        self.press = None
        self.circle.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.circle.figure.canvas.mpl_disconnect(self.cidpress)
        self.circle.figure.canvas.mpl_disconnect(self.cidrelease)
        self.circle.figure.canvas.mpl_disconnect(self.cidmotion)

In [5]:
gca = plt.gca()
nodes = np.array([])
edges = np.array([])

for i in range(n):
    circle = plt.Circle((Xs[i], Ys[i]), radius=0.2, fc='y',label=str(i))
    gca.add_patch(circle)
    nodes = np.append(nodes,circle)

for e in edge:
    line = plt.Line2D((Xs[e[0]-1], Xs[e[1]-1]), (Ys[e[0]-1],Ys[e[1]-1]), lw=1)
    gca.add_line(line)
    edges = np.append(edges,line)
    
dcs = []
for node in nodes:
    dc = DraggableCircle(node)
    dc.connect()
    dcs.append(dc)

plt.axis('scaled')
plt.show()

event contains (-0.48834002780152058, 0.28108480633993033)


NameError: name 'p' is not defined